# Building Network

In this notebook the network according to retweets/mentions/replys will be build and visualized

In [1]:
import pandas as pd
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.dates as md

from itertools import count
from collections import Counter
import matplotlib

import networkx as nx

from fa2 import ForceAtlas2

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
import sys

def parse(f,chunksize = 100000, cols = None, test_bool = False, dtype = None):
    lines = 9041*1000
    for i,gm_chunk in enumerate(pd.read_csv("../ira_tweets_csv_hashed.csv",chunksize = chunksize, usecols = cols, dtype = dtype)):
        f(gm_chunk)
        if test_bool:
            print('Done')
            break
        j = i*chunksize/lines
        sys.stdout.write('\r')
        # the exact output you're looking for:
        bar_int = int(j*20+1)
        sys.stdout.write("[%-20s] %d%%" % ('='*bar_int, int(j*100+1)))
        sys.stdout.flush()

In [3]:
global df_test
def f(chunk):
    global df_test
    df_test = chunk

parse(f, 2, test_bool = True)

df_test[:1].to_string()

FileNotFoundError: File b'../ira_tweets_csv_hashed.csv' does not exist

In [ ]:
global G
G = nx.Graph()
hashtag_set = set()
def f(chunk):
    for uid, fc, lang in zip(chunk['userid'], chunk['follower_count'], chunk['account_language']):
        G.add_node(uid, lang = lang)
            
parse(f, 40000,cols = ['userid', 'follower_count', 'account_language'], test_bool = False)    

In [ ]:
global nset
nset = set(G.nodes())

In [ ]:
cols = ["in_reply_to_userid", "userid", 'retweet_userid', 'user_mentions']

global id_rep_id
id_rep_id = defaultdict(int)
id_retw_id = defaultdict(int)
id_ment_id = defaultdict(int)


def f(chunk):
    #rt_ids_set = rt_ids_set.union(chunk['retweet_tweetid'])
    for uid, rpid,rtwid,mentids in zip(chunk['userid'], chunk['in_reply_to_userid'], chunk['retweet_userid'], chunk['user_mentions']):
        if rpid in nset and uid in nset:
            id_rep_id[(uid, rpid)] += 1
        if rtwid in nset and uid in nset:
            id_retw_id[(uid, rtwid)] += 1
        if mentids == mentids:
            mentids = mentids.replace("[", '').replace("]", '').split(',')
            for mentid in mentids:
                if uid in nset and mentid in nset:
                    id_ment_id[(uid,mentid)] += 1            

parse(f, 100000, cols = cols, test_bool = False)   

In [ ]:
total_edge_dict = defaultdict(int)

for e,w in id_retw_id.items():
    total_edge_dict[e[0],e[1]] += w
    #G.add_edge(e[0], e[1], weight = w)
for e,w in id_rep_id.items():
    #G.add_edge(e[0], e[1], weight = w)
    total_edge_dict[e[0],e[1]] += w
for e,w in id_ment_id.items(): 
    total_edge_dict[e[0],e[1]] += w
    #G.add_edge(e[0], e[1], weight = w)

In [ ]:
for e,w in total_edge_dict.items():
    G.add_edge(e[0], e[1], weight = w)

In [ ]:
comp_G = list(nx.connected_component_subgraphs(G))

In [ ]:
sorted([len(c.nodes()) for c in comp_G])[:-20:-1]

In [ ]:
H = comp_G[0]
pos = nx.spring_layout(H.to_undirected())

In [ ]:
nx.draw(H, pos = pos, node_size = 1)

In [ ]:
forceatlas2 = ForceAtlas2(
                          # Behavior alternatives
                          outboundAttractionDistribution=True,  # Dissuade hubs
                          edgeWeightInfluence= 0.5,

                          # Performance
                          jitterTolerance=1., # Tolerance
                          barnesHutOptimize=True,
                          barnesHutTheta=0.2,

                          # Tuning
                          scalingRatio=2.,
                          strongGravityMode=False,
                          gravity=1,

                          # Log
                          verbose=True)

positions = forceatlas2.forceatlas2_networkx_layout(H.to_undirected(), pos=None, iterations = 100)

In [ ]:
nx.draw(H, pos = positions, node_size = 1)